In [1]:
# import the necessary libraries 
import pandas as pd
import numpy as np
import re

In [2]:
job = pd.read_csv(r"C:\Users\bhard\Downloads\indeed_job_data.csv")

In [3]:
job.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,NaN,Full-time,PostedJust posted,Train volunteers on the use of M&E frameworks ...
1,QA Tester - Intern,mDoc Healthcare,Lagos,NaN,NaN,EmployerActive 6 days ago,Advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,NaN,NaN,EmployerActive 5 days ago,Implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,"₦150,000 - ₦200,000 a month",Full-time,PostedPosted 2 days ago,Analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,NaN,NaN,PostedPosted 1 day ago,Analyse business information/ data to identify...


In [4]:
# Display information about our dataset 
job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        203 non-null    object
 1   Company      200 non-null    object
 2   Location     203 non-null    object
 3   Salary       15 non-null     object
 4   Job Type     101 non-null    object
 5   Date Posted  203 non-null    object
 6   Summary      203 non-null    object
dtypes: object(7)
memory usage: 11.2+ KB


Data Cleaning
Data Cleaning Tasks
Handle Missing Values: Identify and handle missing values in the dataset, especially in columns such as Salary, Company, and Job Type

Extract Numeric Values from Salary: Extract numeric values from the Salary column and convert them into a uniform format, removing currency symbols, text or whitespace as well as dealing with salary range.

Replace inconsistent values in the 'Job Type' column

Parse Date Posted: Parse the Date Posted column to convert dates into a consistent format (e.g., YYYY-MM-DD)

Standardize Summary column: Standardize Summary column to ensure consistency, such as removing leading/trailing whitespace, and fixing any encoding issues (e.g., fixing the characters like â€¦)

#1. Handle missing values

In [5]:
# Identify missing values
missing_values = job.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 Title            0
Company          3
Location         0
Salary         188
Job Type       102
Date Posted      0
Summary          0
dtype: int64


In [7]:
# Fill missing Company values with a default value
job['Company'].fillna('Unknown', inplace=True)

# Fill missing Job Type values with a default value
job['Job Type'].fillna('Unknown', inplace=True) 

#2. Extract Numeric Values from Salary

In [8]:
job["Salary"].unique()

array([nan, '₦150,000 - ₦200,000 a month', '₦100,000 - ₦150,000 a month',
       '₦200,000 a month', '₦70,000 - ₦150,000 a month',
       '₦90,000 - ₦130,000 a month', '₦300,000 - ₦350,000 a month',
       '₦450,000 - ₦600,000 a month', '₦200,000 - ₦250,000 a month',
       '₦220,000 a month', '₦60,000 - ₦80,000 a month'], dtype=object)

In [11]:
# Remove naira symbols, commas, whitespace and non-numeric characters
job['Salary'] = job['Salary'].str.replace(r'[₦,a-zA-Z\s]', '', regex=True)


In [12]:
# Convert values in range to average
def convert_to_average(salary):
    if isinstance(salary, str) and '-' in salary:
        # Extract lower and upper range values
        lower, upper = map(int, salary.split('-'))
        return (lower + upper) / 2
    else:
        return salary

job['Salary'] = job['Salary'].apply(convert_to_average)

In [13]:
# Convert to numeric
job['Salary'] = job['Salary'].astype(float)

In [15]:
# Impute missing salary values with the median of salary
job['Salary'] = job['Salary'].fillna(job['Salary'].median())


In [16]:
job["Salary"].unique()

array([175000., 125000., 200000., 110000., 325000., 525000., 225000.,
       220000.,  70000.])

#For the Salary column, the data cleaning processes we performed are;

Retrieve unique values from the 'Salary' column to understand the salary data distribution.

Clean the 'Salary' column by removing Naira symbols, commas, whitespace, and non-numeric characters using regular expressions.

Convert salary ranges to their average values to provide a more accurate representation.

Convert the cleaned salary values to numeric format by casting them to float.

Fill any remaining missing values in the 'Salary' column with the median salary.

Finally, retrieve the unique values from the 'Salary' column again to verify the cleaning and conversion process.

In [17]:
# Identify missing values
missing_values = job.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 Title          0
Company        0
Location       0
Salary         0
Job Type       0
Date Posted    0
Summary        0
dtype: int64


#3. Replace inconsistent values in the 'Job Type' column

In [19]:
job['Job Type'].unique()

array(['Full-time', 'Unknown', 'Temporary', 'Full-time +1', 'Contract',
       'Permanent'], dtype=object)

In [20]:
# Replace inconsistent values in the 'Job Type' column
job['Job Type'] = job['Job Type'].replace({'Contract +1': 'Contract', 'Full-time +1': 'Full-time'})

#4. Parse Date Posted

In [21]:
# Clean Date Posted column
job['Date Posted'] = job['Date Posted'].str.replace('Posted', '')  # Remove "Posted" phrase
job['Date Posted'] = job['Date Posted'].str.replace('EmployerActive', '')  # Remove "EmployerActive" phrase
job['Date Posted'] = job['Date Posted'].str.replace('Today', '0 days ago')  # Replace "Today" with 0 days ago

In [23]:
def convert_to_date(value):
    try:
        if 'yesterday' in value.lower():
            return pd.Timestamp.now().normalize() - pd.Timedelta(days=1)
        elif 'today' in value.lower():
            return pd.Timestamp.now().normalize()
        elif '+' in value:
            days = int(value.split('+')[0])
            return pd.Timestamp.now().normalize() - pd.Timedelta(days=days)
        else:
            days = int(value.split()[0])
            return pd.Timestamp.now().normalize() - pd.Timedelta(days=days)
    except (ValueError, IndexError):
        return pd.NaT


#Standardize Summary column

In [24]:
# Function to remove special characters and non-letter characters
def clean_summary(summary):
    # Define regular expression pattern to match non-letter characters
    pattern = r'[^a-zA-Z\s]'
    # Replace non-letter characters with a space
    cleaned_summary = re.sub(pattern, ' ', summary)
    # Remove extra whitespaces
    cleaned_summary = ' '.join(cleaned_summary.split())
    return cleaned_summary

# Apply the clean_summary function to the Summary column
job['Summary'] = job['Summary'].apply(clean_summary)

In [25]:
job.head()

,Title,Company,Location,Salary,Job Type,Date Posted,Summary
0,"PO, Research & Data Analyst",The Elevation Church,Nigeria,175000.0,Full-time,Just posted,Train volunteers on the use of M E frameworks ...
1,QA Tester - Intern,mDoc Healthcare,Lagos,175000.0,Unknown,6 days ago,Advanced knowledge of data security and encryp...
2,Data Analyst - Healthcare,eMedicStore,Lagos,175000.0,Unknown,5 days ago,Implement new data analysis methodologies and ...
3,HR Analyst,Jobrole Consulting Limited,Ikeja,175000.0,Full-time,2 days ago,Analyze and interpret data to generate actiona...
4,Business Intelligence Analyst,TSL Metroline Limited,Lagos,175000.0,Unknown,1 day ago,Analyse business information data to identify ...


In [26]:
job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        203 non-null    object 
 1   Company      203 non-null    object 
 2   Location     203 non-null    object 
 3   Salary       203 non-null    float64
 4   Job Type     203 non-null    object 
 5   Date Posted  203 non-null    object 
 6   Summary      203 non-null    object 
dtypes: float64(1), object(6)
memory usage: 11.2+ KB


In [27]:
# Save the cleaned DataFrame to a CSV file

job.to_csv("cleaned_job_data.csv", index=False)